Link to Datatset

https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv

Follow through tutorial

https://analyticsindiamag.com/guide-to-tensorflow-extendedtfx-end-to-end-platform-for-deploying-production-ml-pipelines/

In [2]:
import tensorflow as tf
from tfx.examples import chicago_taxi_pipeline
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Evaluator
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import tensorflow_transform as tft
from tfx import v1 as tfx
import tensorflow_model_analysis as tfma

import os, tempfile, absl, urllib, pprint
import taxi_constants, taxi_trainer # TAXI_CONSTANTS for preprocessing & Trainer for training

Setup the pipeline paths

In [3]:
# this is the dir containing the TFX chicago example
_taxi_root = chicago_taxi_pipeline.__path__[0]

# path to push model for serving
_serving_model_dir = os.path.join(tempfile.mkdtemp(), 'serving_model/taxi_simple')

# set up logging
absl.logging.set_verbosity(absl.logging.INFO)

In [6]:
print("Taxi root: {}".format(_taxi_root))
print("Serving model Directory: {}".format(_serving_model_dir))
pp = pprint.PrettyPrinter()

Taxi root: C:\Users\DELL\AppData\Local\Programs\Python\Python37\lib\site-packages\tfx\examples\chicago_taxi_pipeline
Serving model Directory: C:\Users\DELL\AppData\Local\Temp\tmp5_csftst\serving_model/taxi_simple


In [7]:
# download the dataset
_data_root = tempfile.mkdtemp(prefix='tfx-data')
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'
_data_filepath = os.path.join(_data_root, 'data.csv')
urllib.request.urlretrieve(DATA_PATH, _data_filepath)


('C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-data6s_8m_x0\\data.csv',
 <http.client.HTTPMessage at 0x1c94e2db4e0>)

In [9]:
# initialize the Interactive-Context
context = InteractiveContext()


Ingest the data

In [10]:
# ingests the data
example_gen = CsvExampleGen(input_base=_data_root)
context.run(example_gen)


INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data C:\Users\DELL\AppData\Local\Temp\tfx-data6s_8m_x0\* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\\CsvExampleGen\\examples\\1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:1922812,xor_checksum:1643110451,sum_checksum:1643110451"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [11]:
# inspect the artifact of EXAMPLE_GEN
artifact = example_gen.outputs['examples'].get()[0]

print("Artifact Split names: {}".format(artifact.split_names))
print("Artifact URI: {}".format(artifact.uri))

Artifact Split names: ["train", "eval"]
Artifact URI: C:\Users\DELL\AppData\Local\Temp\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\CsvExampleGen\examples\1


Analyze the data, using StatisticsGen

In [12]:
# the output of STEP 1 is the input for this
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen) 

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to C:\Users\DELL\AppData\Local\Temp\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\StatisticsGen\statistics\2\Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to C:\Users\DELL\AppData\Local\Temp\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\StatisticsGen\statistics\2\Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 16
        uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\\StatisticsGen\\statistics\\2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [13]:
# visualize the analysis provided by the StatisticsGen
context.show(statistics_gen.outputs['statistics'])

Generate SchemaGen from the statistics_gen, which is used to create summary data.

In [14]:
# generates Schema for the dataset by setting boundaries, types and properties of the features of the dataset
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'], infer_feature_shape=False)
context.run(schema_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to C:\Users\DELL\AppData\Local\Temp\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\SchemaGen\schema\3\schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 18
        uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\\SchemaGen\\schema\\3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [15]:
# visualize the generated stats
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'company',STRING,required,,'company'
'dropoff_census_tract',INT,required,,-
'dropoff_community_area',INT,required,,-
'dropoff_latitude',FLOAT,required,,-
'dropoff_longitude',FLOAT,required,,-
'fare',FLOAT,required,single,-
'payment_type',STRING,required,single,'payment_type'
'pickup_census_tract',INT,required,,-
'pickup_community_area',INT,required,,-


C:\Users\DELL\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow_data_validation\utils\display_util.py:180: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'company',"'0118 - 42111 Godfrey S.Awir', '1085 - 72312 N and W Cab Co', '2192 - 73487 Zeymane Corp', '2733 - 74600 Benny Jona', '3011 - 66308 JBL Cab Inc.', '3152 - 97284 Crystal Abernathy', '3201 - C&D Cab Co Inc', '3201 - CID Cab Co Inc', '3253 - 91138 Gaither Cab Co.', '3319 - CD Cab Co', '3385 - 23210 Eman Cab', '3385 - Eman Cab', '3623 - 72222 Arrington Enterprises', '3897 - 57856 Ilie Malec', '4053 - 40193 Adwar H. Nikola', '4197 - 41842 Royal Star', '4197 - Royal Star', '4615 - 83503 Tyrone Henderson', '4615 - Tyrone Henderson', '4623 - Jay Kim', '5006 - 39261 Salifu Bawa', '5074 - 54002 Ahzmi Inc', '5074 - Ahzmi Inc', '5129 - 87128', '5129 - 98755 Mengisti Taxi', '585 - 88805 Valley Cab Co', '5864 - Thomas Owusu', '5874 - 73628 Sergey Cab Corp.', '5874 - Sergey Cab Corp.', '5997 - 65283 AW Services Inc.', '6488 - 83287 Zuha Taxi', '6574 - Babylon Express Inc.', '6742 - 83735 Tasha ride inc', 'Blue Ribbon Taxi Association Inc.', 'C & D Cab Co Inc', 'Chicago Elite Cab Corp.', 'Chicago Elite Cab Corp. (Chicago Carriag', 'Chicago Medallion Leasing INC', 'Chicago Medallion Management', 'Choice Taxi Association', 'Dispatch Taxi Affiliation', 'KOAM Taxi Association', 'Northwest Management LLC', 'Taxi Affiliation Services', 'Top Cab Affiliation', '0694 - 59280 Chinesco Trans Inc', '2092 - 61288 Sbeih company', '2192 - Zeymane Corp', '2809 - 95474 C & D Cab Co Inc.', '2823 - 73307 Seung Lee', '3094 - 24059 G.L.B. Cab Co', '3897 - Ilie Malec', '4053 - Adwar H. Nikola', '5006 - Salifu Bawa', '5129 - Mengisti Taxi', '5724 - KYVI Cab Inc', '585 - Valley Cab Co', '5864 - 73614 Thomas Owusu', '5997 - AW Services Inc.', '6057 - 24657 Richard Addo', '6743 - Luhak Corp'"
'payment_type',"'Cash', 'Credit Card', 'Dispute', 'No Charge', 'Pcard', 'Unknown', 'Prcard'"


This looks for anomalies and null values in the dataset.

In [16]:
# takes StatisticsGen and SchemaGen as inputs
example_validator = ExampleValidator(statistics=statistics_gen.outputs['statistics'], 
                    schema=schema_gen.outputs['schema'])

context.run(example_validator) # no anomalies in our dataset

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to C:\Users\DELL\AppData\Local\Temp\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\ExampleValidator\anomalies\4\Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to C:\Users\DELL\AppData\Local\Temp\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\ExampleValidator\anomalies\4\Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 20
        uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\\ExampleValidator\\anomalies\\4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [17]:
# visualize the validator
# no anomalies found
context.show(example_validator.outputs['anomalies'])

C:\Users\DELL\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow_data_validation\utils\display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


Transform with TF-Transform, which does the feature/data engineering

In [18]:
# takes the Example Gen, Schema Gen and UDF Code that does the transformation
# the UDF is contained in a *TAXI_CONSTANTS.PY* file
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath("taxi_constants.py")
)

context.run(transform)
# outputs the Transform graph and the Transformed examples

INFO:absl:Generating ephemeral wheel package for 'c:\\Users\\DELL\\Desktop\\Learning projects\\Learning ML pipelines\\taxi_constants.py' (including modules: ['ML_pipelines', 'taxi_constants']).
INFO:absl:User module package has hash fingerprint version 0b6ee190bd3ed33097a596f51421518cb015ea5aa16cbc2902194f8f49fc710d.
INFO:absl:Executing: ['C:\\Users\\DELL\\AppData\\Local\\Programs\\Python\\Python37\\python.exe', 'C:\\Users\\DELL\\AppData\\Local\\Temp\\tmp7hy_86xl\\_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', 'C:\\Users\\DELL\\AppData\\Local\\Temp\\tmpapjiqw7h', '--dist-dir', 'C:\\Users\\DELL\\AppData\\Local\\Temp\\tmp1l6yg2bg']
INFO:absl:Successfully built user code wheel distribution at 'C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\\_wheels\\tfx_user_code_Transform-0.0+0b6ee190bd3ed33097a596f51421518cb015ea5aa16cbc2902194f8f49fc710d-py3-none-any.whl'; target user module is 'taxi_constants'.
INFO:absl:Full user module path is 'ta

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.
INFO:absl:Feature company has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature dropoff_census_tract has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature dropoff_community_area has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature dropoff_latitude has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature dropoff_longitude has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature fare has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature payment_type has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature pickup_census_tract has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature pickup_community_area has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature pickup_latitude has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature pickup_longitude has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature tips has no shape. Setting to VarLenSparseTensor.
INFO:absl:Fea

INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\Transform\transform_graph\5\.temp_path\tftransform_tmp\2b7c07591f0e4bea8af75a94fe7e2a30\assets


INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\Transform\transform_graph\5\.temp_path\tftransform_tmp\2b7c07591f0e4bea8af75a94fe7e2a30\assets


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.
INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 5
        type_id: 22
        uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\\Transform\\transform_graph\\5"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 22
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 6
        type_id: 14
        uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\\Transform\\transformed_examples\\5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 7
        type_id: 23
        uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\\Transform\\updated_analyzer_cache\\5"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 8
        type_id: 18
        uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\\Transform\\pre_transform_schema\\5"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_

The TF-TRANSFORM outputs two artifacts. The transform graph which can perform the preprocessing ops both in
 training & serving. Secondly, transformed exmaples which is preprocessed training and evaluation data.

In [19]:
transform.outputs

{'transform_graph': Channel(
     type_name: TransformGraph
     artifacts: [Artifact(artifact: id: 5
 type_id: 22
 uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\\Transform\\transform_graph\\5"
 custom_properties {
   key: "name"
   value {
     string_value: "transform_graph"
   }
 }
 custom_properties {
   key: "producer_component"
   value {
     string_value: "Transform"
   }
 }
 custom_properties {
   key: "state"
   value {
     string_value: "published"
   }
 }
 custom_properties {
   key: "tfx_version"
   value {
     string_value: "1.3.0"
   }
 }
 state: LIVE
 , artifact_type: id: 22
 name: "TransformGraph"
 )]
     additional_properties: {}
     additional_custom_properties: {}
 ),
 'transformed_examples': Channel(
     type_name: Examples
     artifacts: [Artifact(artifact: id: 6
 type_id: 14
 uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\\Transform\\transformed_examples\\5"

In [20]:
# taking a Peek at the Transformed graph artifacts
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

# the metadata dir contains the Schema of the original data
# transformed dir metadata contains the Schema of the preprocessed data
# transformed_fn dir contains the actual preprocessing graph

['metadata', 'transformed_metadata', 'transform_fn']

In [36]:
# take a Peek at the first 3 transformed examples
# get the dir of the  preprocessed examples
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split_train')

# get the list of outputs in this directory
# tfrecord_filenames =  [os.path.join(train_uri, name) for name in os.listdir(train_uri)]

# create a TFRecordDataset to read these files
transformed_file = 'transformed_examples-00000-of-00001.gz'
dataset = tf.data.TFRecordDataset(transformed_file, compression_type='GZIP')

# iterate over the first 3 records and decode them
for tfrecord in dataset.take(3):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)
    



features {
  feature {
    key: "company"
    value {
      int64_list {
        value: 8
      }
    }
  }
  feature {
    key: "dropoff_census_tract"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "dropoff_community_area"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "dropoff_latitude"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "dropoff_longitude"
    value {
      int64_list {
        value: 9
      }
    }
  }
  feature {
    key: "fare"
    value {
      float_list {
        value: 0.061060599982738495
      }
    }
  }
  feature {
    key: "payment_type"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "pickup_census_tract"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "pickup_community_area"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "

Trainer Component

In [41]:
trainer = Trainer(
    module_file='taxi_trainer.py',
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=tfx.proto.TrainArgs(num_steps=10000),
    eval_args=tfx.proto.EvalArgs(num_steps=5000)
)

context.run(trainer)

INFO:absl:Generating ephemeral wheel package for 'c:\\Users\\DELL\\Desktop\\Learning projects\\Learning ML pipelines\\taxi_trainer.py' (including modules: ['ML_pipelines', 'taxi_constants', 'taxi_trainer']).
INFO:absl:User module package has hash fingerprint version 5c15dd8192e00706e0cadf4ec1de590789321fd9387438746b5d71246a062bcf.
INFO:absl:Executing: ['C:\\Users\\DELL\\AppData\\Local\\Programs\\Python\\Python37\\python.exe', 'C:\\Users\\DELL\\AppData\\Local\\Temp\\tmpjdito60r\\_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', 'C:\\Users\\DELL\\AppData\\Local\\Temp\\tmplpstmuox', '--dist-dir', 'C:\\Users\\DELL\\AppData\\Local\\Temp\\tmphapwhy0t']
INFO:absl:Successfully built user code wheel distribution at 'C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\\_wheels\\tfx_user_code_Trainer-0.0+5c15dd8192e00706e0cadf4ec1de590789321fd9387438746b5d71246a062bcf-py3-none-any.whl'; target user module is 'taxi_trainer'.
INFO:absl:Full user module p

10000/10000 [==============================] - 49s 5ms/step - loss: 0.2387 - binary_accuracy: 0.8603 - val_loss: 0.2223 - val_binary_accuracy: 0.8750
INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert



INFO:absl:eval_transformed_features = {'trip_start_day': <tf.Tensor 'transform_features_layer/StatefulPartitionedCall:14' shape=(None,) dtype=int64>, 'pickup_longitude': <tf.Tensor 'transform_features_layer/StatefulPartitionedCall:10' shape=(None,) dtype=int64>, 'tips': <tf.Tensor 'transform_features_layer/StatefulPartitionedCall:11' shape=(None,) dtype=int64>, 'trip_start_month': <tf.Tensor 'transform_features_layer/StatefulPartitionedCall:16' shape=(None,) dtype=int64>, 'trip_miles': <tf.Tensor 'transform_features_layer/StatefulPartitionedCall:12' shape=(None,) dtype=float32>, 'dropoff_community_area': <tf.Tensor 'transform_features_layer/StatefulPartitionedCall:2' shape=(None,) dtype=int64>, 'pickup_census_tract': <tf.Tensor 'transform_features_layer/StatefulPartitionedCall:7' shape=(None,) dtype=int64>, 'company': <tf.Tensor 'transform_features_layer/StatefulPartitionedCall:0' shape=(None,) dtype=int64>, 'payment_type': <tf.Tensor 'transform_features_layer/StatefulPartitionedCall:

INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\Trainer\model\6\Format-Serving\assets


INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\Trainer\model\6\Format-Serving\assets
INFO:absl:Training complete. Model written to C:\Users\DELL\AppData\Local\Temp\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\Trainer\model\6\Format-Serving. ModelRun written to C:\Users\DELL\AppData\Local\Temp\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\Trainer\model_run\6
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 13
        type_id: 25
        uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\\Trainer\\model\\6"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 25
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 14
        type_id: 26
        uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\\Trainer\\model_run\\6"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 26
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

Analyze training with TensorBoard

In [42]:
model_artifact_dir = trainer.outputs['model'].get()[0].uri
pp.pprint(os.listdir(model_artifact_dir))
model_dir = os.path.join(model_artifact_dir, 'Format-Serving')
pp.pprint(os.listdir(model_dir))

['Format-Serving']
['assets', 'keras_metadata.pb', 'saved_model.pb', 'variables']


In [44]:
# view the model on TensorBoard
model_run_artifact_dir = trainer.outputs['model_run'].get()[0].uri

%load_ext tensorboard
%tensorboard --logdir {model_run_artifact_dir}

In [45]:
%tensorboard --logdir=model_run_artifact_dir/ --host localhost --port 8088

# enter the link below in browser to view TB
# http://localhost:8088

Evaluator guages the performance metric of the evaluation set through Tensorflow Model Analysis (TFMA).

In [53]:
# spcify the Evaluation configuration
eval_config = tfma.EvalConfig(
    model_specs=[
        # this assumes a serving model with signature 'serving_default'
        tfma.ModelSpec(
            signature_name='serving_default',
            label_key='tips', preprocessing_function_names=['transform_features'],
        )
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(
                    class_name='BinaryAccuracy', threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(lower_bound={'value': 0.5}),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER, absolute={'value': -1e-10}
                    ))
                )
            ] 
        )
    ],
    slicing_specs=[
        # an empty slice specs means the whole dataset
        tfma.SlicingSpec(),
        # then the data is sliced along a FEATURE COLUMN 
        tfma.SlicingSpec(feature_keys=['trip_start_hour'])
    ]
)



# The model resolver is only required if performing model validation in addition to evaluation.
# In this case we validate against the latest blessed model. If no model has been blessed before (as in this case) 
# the evaluator will make our candidate the first blessed model.

model_resolver = tfx.dsl.Resolver(
    strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
    model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
    model_blessing=tfx.dsl.Channel(
        type=tfx.types.standard_artifacts.ModelBlessing
    )
).with_id('latest_blessed_model_resolver')

context.run(model_resolver)

INFO:absl:Running driver for latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 7
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [56]:
# run the Evaluator
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)

context.run(evaluator)


INFO:absl:Running driver for Evaluator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Evaluator
INFO:absl:Nonempty beam arg extra_packages already includes dependency
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "ExampleCount"\n        },\n        {\n          "class_name": "BinaryAccuracy",\n          "threshold": {\n            "change_threshold": {\n              "absolute": -1e-10,\n              "direction": "HIGHER_IS_BETTER"\n            },\n            "value_threshold": {\n              "lower_bound": 0.5\n            }\n          }\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "tips",\n      "preprocessing_function_names": [\n        "transform_features"\n      ],\n      "signature_name": "serving_default"\n    }\n  ],\n  "slicing_specs": [\n    {},\n    {\n      "feature_keys": [\n        "trip_start_hour"\n      ]\n  


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x000001C94EC1C7F0> and <keras.engine.input_layer.InputLayer object at 0x000001C94EA96630>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x000001C94EC1C7F0> and <keras.engine.input_layer.InputLayer object at 0x000001C94EA96630>).
INFO:absl:The 'example_splits' parameter is not set, using 'eval' split.
INFO:absl:Evaluating model.
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "ExampleCount"\n        },\n        {\n          "class_name": "BinaryAccuracy",\n          "threshold": {\n            "change_threshold": {\n              "absolute": -1e-10,\n              "direction": "HIGHER_IS_BETTER"\n            },\n            "value_threshold": {\n              "lower_bound": 0.5\n            }\n          }\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "tips",\n      "preprocessing_function_names": [\n        "transform_features"\n      ],\n      "signature_name": "se


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x000001C96639F400> and <keras.engine.input_layer.InputLayer object at 0x000001C9663E24E0>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x000001C96639F400> and <keras.engine.input_layer.InputLayer object at 0x000001C9663E24E0>).


...


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x000001C907AECCF8> and <keras.engine.input_layer.InputLayer object at 0x000001C900183C88>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x000001C907AECCF8> and <keras.engine.input_layer.InputLayer object at 0x000001C900183C88>).
INFO:absl:Evaluation complete. Results written to C:\Users\DELL\AppData\Local\Temp\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\Evaluator\evaluation\8.
INFO:absl:Checking validation results.


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
INFO:absl:Blessing result True written to C:\Users\DELL\AppData\Local\Temp\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\Evaluator\blessing\8.
INFO:absl:Running publisher for Evaluator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Evaluator
    execution_id: 8
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 15
        type_id: 29
        uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\\Evaluator\\evaluation\\8"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 29
        name: "ModelEvaluation"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 16
        type_id: 30
        uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\\Evaluator\\blessing\\8"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\\Trainer\\model\\6"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 13
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 30
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [57]:
# check the Evaluator outputs
evaluator.outputs

{'evaluation': Channel(
     type_name: ModelEvaluation
     artifacts: [Artifact(artifact: id: 15
 type_id: 29
 uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\\Evaluator\\evaluation\\8"
 custom_properties {
   key: "name"
   value {
     string_value: "evaluation"
   }
 }
 custom_properties {
   key: "producer_component"
   value {
     string_value: "Evaluator"
   }
 }
 custom_properties {
   key: "state"
   value {
     string_value: "published"
   }
 }
 custom_properties {
   key: "tfx_version"
   value {
     string_value: "1.3.0"
   }
 }
 state: LIVE
 , artifact_type: id: 29
 name: "ModelEvaluation"
 )]
     additional_properties: {}
     additional_custom_properties: {}
 ),
 'blessing': Channel(
     type_name: ModelBlessing
     artifacts: [Artifact(artifact: id: 16
 type_id: 30
 uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\\Evaluator\\blessing\\8"
 custom_properties {
   key: 

In [60]:
context.show(evaluator.outputs['evaluation'])

In [61]:
# Get the TFMA output result path and load the result.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

# Show data sliced along feature column trip_start_hour.
tfma.view.render_slicing_metrics(tfma_result, slicing_column='trip_start_hour')

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'trip_start_hour:19',…

PUSHER component, pushes the model to teh REST API for serving

In [62]:
pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher)

INFO:absl:Running driver for Pusher
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Pusher
INFO:absl:Model version: 1643140255
INFO:absl:Model written to serving path C:\Users\DELL\AppData\Local\Temp\tmp5_csftst\serving_model/taxi_simple\1643140255.
INFO:absl:Model pushed to C:\Users\DELL\AppData\Local\Temp\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\Pusher\pushed_model\9.
INFO:absl:Running publisher for Pusher
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Pusher
    execution_id: 9
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 17
        type_id: 32
        uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\\Pusher\\pushed_model\\9"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tmp5_csftst\\serving_model/taxi_simple\\1643140255"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1643140255"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 32
        name: "PushedModel"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [63]:
pusher.outputs

{'pushed_model': Channel(
     type_name: PushedModel
     artifacts: [Artifact(artifact: id: 17
 type_id: 32
 uri: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tfx-interactive-2022-01-25T12_35_06.757170-xluoreeq\\Pusher\\pushed_model\\9"
 custom_properties {
   key: "name"
   value {
     string_value: "pushed_model"
   }
 }
 custom_properties {
   key: "producer_component"
   value {
     string_value: "Pusher"
   }
 }
 custom_properties {
   key: "pushed"
   value {
     int_value: 1
   }
 }
 custom_properties {
   key: "pushed_destination"
   value {
     string_value: "C:\\Users\\DELL\\AppData\\Local\\Temp\\tmp5_csftst\\serving_model/taxi_simple\\1643140255"
   }
 }
 custom_properties {
   key: "pushed_version"
   value {
     string_value: "1643140255"
   }
 }
 custom_properties {
   key: "state"
   value {
     string_value: "published"
   }
 }
 custom_properties {
   key: "tfx_version"
   value {
     string_value: "1.3.0"
   }
 }
 state: LIVE
 , artifact_type: id: 32
 name: "Pushed

In [64]:
push_uri = pusher.outputs['pushed_model'].get()[0].uri
model = tf.saved_model.load(push_uri)

for item in model.signatures.items():
  pp.pprint(item)

('serving_default',
 <ConcreteFunction signature_wrapper(*, examples) at 0x1C984339DD8>)
('transform_features',
 <ConcreteFunction signature_wrapper(*, examples) at 0x1C9831E6CC0>)
